In [111]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import seaborn as sns
from tabulate import tabulate
from astropy.time import Time
from tqdm import tqdm
from scipy.spatial.distance import pdist, squareform
import numpy as np
from astropy.time import Time
import datetime
from collections import defaultdict
from scipy.spatial import cKDTree


In [127]:
df_vlass = pd.read_pickle('/datax/scratch/ellambishop/temp_hits_organized/non_vlass_other.pkl')
#df_vlass.groupby('tstart').size()
df_vlass.columns
len(df_vlass)
#df_vlass['freq_has_incoherent_counterpart']

11482838

In [113]:
#clean out initial drift rates=0 and snr > 16
def set_filter(file, drift_max=0):
    # Load and slice dataset
    global df_new
    df = file
   # print(df.columns)
    #small_df = df[start:stop]

    # Select relevant columns
    columns = ['file_uri', 'observation_id','source_name', 'beam_id', 'ra_hours', 'dec_degrees', 'tstart',
               'signal_frequency', 'signal_beam', 'signal_drift_rate', 'signal_snr',
               'signal_power', 'signal_incoherent_power', 'signal_num_timesteps','freq_has_incoherent_counterpart', 'freq_has_phase_center_counterpart' ]
    df_new = df[columns]

    # Apply filtering thresholds
    df_new = df_new[(df_new['signal_drift_rate'] != drift_max)]
    df_new = df_new[(df_new['signal_snr'])<=16]
    df_new['signal_frequency'] = df_new['signal_frequency'].round(3)    
    
    #print(df_new.groupby('signal_frequency').size())
    return df_new

#set_filter(df_vlass)


In [114]:
#rfi filtering based on time step- snr ranges and frequency overlap flagging
def processing(df):
    """
    Processes filtered DataFrame by grouping on (file_uri, observation_id),
    flags RFI based on conditions, and returns the flagged DataFrame.
    """
    # Apply initial filtering
    df_new = set_filter(df)

    # Group by file_uri and observation_id
    grouped = df_new.groupby(['file_uri', 'observation_id'])
    flagged_dfs = []

    for (file_uri, obs_id), group_df in grouped:
        group_df = group_df.copy()
        group_df['rfi_flag'] = False
        group_df['flag_strength'] = 0

        # Conditions to drop hits
        cond1 = (
            (group_df['signal_num_timesteps'] >= 16) &
            (group_df['signal_num_timesteps'] <= 64) &
            (group_df['signal_snr'] > 10)
        )
        cond2 = (
            (group_df['signal_num_timesteps'] < 16) &
            (group_df['signal_snr'] > 15)
        )

        drop_mask = cond1 | cond2
        group_df = group_df.loc[~drop_mask]

        flagged_dfs.append(group_df)

    full_df = pd.concat(flagged_dfs, ignore_index=True)
    return full_df

#full_df = pd.concat(flagged_dfs, ignore_index=True)
#processing()


In [126]:
full_df= processing(df_vlass)
len(full_df)

804791

In [116]:
#convert time to dates and flag the different NRAO configurations 

# Extended VLA configuration date ranges to fill gaps

vla_config_ranges = [
    (datetime.date(2023, 1, 19), datetime.date(2023, 5, 29), 'B'),
    (datetime.date(2023, 6, 2),  datetime.date(2023, 6, 19), 'BnA'),
    (datetime.date(2023, 6, 30), datetime.date(2023, 10, 2), 'A'),
    (datetime.date(2023, 10, 3), datetime.date(2023, 10, 19), 'A'),
    (datetime.date(2023, 10, 20), datetime.date(2024, 1, 15), 'D'),
    (datetime.date(2024, 1, 16), datetime.date(2024, 1, 24), 'D'),   # patch
    (datetime.date(2024, 1, 25), datetime.date(2024, 5, 7), 'C'),
    (datetime.date(2024, 5, 8),  datetime.date(2024, 9, 16), 'B'),
    (datetime.date(2024, 9, 17), datetime.date(2024, 10, 7), 'BnA'),
    (datetime.date(2024, 10, 8), datetime.date(2025, 2, 3), 'A'),
    (datetime.date(2025, 2, 4), datetime.date(2025, 2, 24), 'A'),     # new patch
    (datetime.date(2025, 2, 25), datetime.date(2025, 5, 12), 'D'),
    (datetime.date(2025, 5, 13), datetime.date(2025, 5, 21), 'C'),    # patch
    (datetime.date(2025, 5, 22), datetime.date(2025, 8, 18), 'C'),
    (datetime.date(2025, 8, 19), datetime.date(2025, 9, 2), 'B'),     # patch
    (datetime.date(2025, 9, 3),  datetime.date(2026, 1, 20), 'B'),
]


dates = []
configs = []

for t in full_df['tstart']:
    try:
        time = Time(t, format='mjd')
        date_str = time.to_value('iso', subfmt='date')
        date_obj = datetime.datetime.strptime(date_str, "%Y-%m-%d").date()
        dates.append(date_str)

        matched_config = 'Unknown'
        for start, end, config in vla_config_ranges:
            if start <= date_obj <= end:
                matched_config = config
                break
        configs.append(matched_config)

    except Exception as e:
        print(f"[!] Error parsing tstart={t}: {e}")
        dates.append('Invalid')
        configs.append('Unknown')

full_df['date'] = dates
full_df['config'] = configs

print(full_df['config'].value_counts())

config
D      493905
A      220656
B       47747
C       41447
BnA      1036
Name: count, dtype: int64


In [117]:
# calculate and assign the size of the beam for each hit 
def assign_beam_size_arcsec(full_df):
    # Define band frequency ranges (MHz)
    band_ranges_mhz = {
        'S':  (2000, 4000),   # 3.0 GHz
        'C':  (4000, 8000),   # 6.0 GHz
        'X':  (8000, 12000),  # 10 GHz
        'Ku': ()
    }

    # Beam size per band and configuration (arcsec)
    beam_resolutions = {
        'S':  {'D': 23,   'C': 7.0,  'B': 2.1,  'A': 0.65},
        'C':  {'D': 12,   'C': 3.5,  'B': 1.0,  'A': 0.33},
        'X':  {'D': 7.2,  'C': 2.1,  'B': 0.6,  'A': 0.20},
    }

    # Build band assignment conditions
    conditions = [
        (full_df['signal_frequency'] >= 2000) & (full_df['signal_frequency'] < 4000),
        (full_df['signal_frequency'] >= 4000) & (full_df['signal_frequency'] < 8000),
        (full_df['signal_frequency'] >= 8000) & (full_df['signal_frequency'] < 12000),
    ]
    band_labels = ['S', 'C', 'X']

    # Assign bands
    full_df['band'] = np.select(conditions, band_labels, default=None)

    # Create lookup dictionary
    beam_lookup = {
        (band, cfg): size
        for band, cfgs in beam_resolutions.items()
        for cfg, size in cfgs.items()
    }

    # Combine band and config as key
    full_df['beam_key'] = list(zip(full_df['band'], full_df['config']))

    # Assign beam size
    full_df['beam_size_arcsec'] = full_df['beam_key'].map(beam_lookup)

    return full_df
full_df= assign_beam_size_arcsec(full_df)

In [118]:
def split_hits_by_beam(df):
    import pandas as pd
    from collections import defaultdict

    grouped_dfs = defaultdict(list)

    for fov, group in df.groupby('file_uri'):
        # Group by frequency instead of hit ID
        beam_counts = group.groupby('signal_frequency')['signal_beam'].nunique()

        for freq, n_beams in beam_counts.items():
            if n_beams > 5:  # Safety check
                continue

            hit_group = group[group['signal_frequency'] == freq]
            key = f'{n_beams}_beams'
            grouped_dfs[key].append(hit_group)

    # Combine all groups into DataFrames
    for key in grouped_dfs:
        grouped_dfs[key] = pd.concat(grouped_dfs[key], ignore_index=True)
        #print(f"Stored {len(grouped_dfs[key])} hits in memory as '{key}'")

    return grouped_dfs


In [119]:
df_groups = split_hits_by_beam(full_df)
# Combine 1- and 2-beam hits into a single DataFrame
df_1_2_beam = pd.concat([df_groups['1_beams'], df_groups['2_beams']], ignore_index=True)

# Combine 3-, 4-, and 5-beam hits into a single DataFrame
df_3_4_5_beam = pd.concat(
    [df_groups['3_beams'], df_groups['4_beams'], df_groups['5_beams']],
    ignore_index=True
)


In [120]:
#flag overlapping signals in spacially separated beams as rfi, same file uri/fov 
def flag_local_rfi(df, area_multiplier=3, beam_threshold_fraction=0.75):
    from scipy.spatial import ConvexHull
    import numpy as np

    local_flag_count = 0
    df['rfi_flag_local'] = False

    unique_freqs = sorted(df['signal_frequency'].unique())

    # ⚡ Precompute beam counts per FOV
    beam_count_by_fov = df.groupby('file_uri')['signal_beam'].nunique().to_dict()

    for freq in unique_freqs:
        freq_subset = df[df['signal_frequency'] == freq]

        for fov in freq_subset['file_uri'].unique():
            subset = freq_subset[freq_subset['file_uri'] == fov]
            unique_beams = subset['signal_beam'].unique()

            total_beams = beam_count_by_fov.get(fov, 0)
            min_required_beams = max(3, int(beam_threshold_fraction * total_beams))

            if len(unique_beams) < min_required_beams or len(subset) < 3:
                continue

            ra0, dec0 = np.mean(subset['ra_hours']), np.mean(subset['dec_degrees'])
            ra_offsets = (subset['ra_hours'] - ra0) * 3600 * np.cos(np.deg2rad(dec0))
            dec_offsets = (subset['dec_degrees'] - dec0) * 3600
            points = np.vstack([ra_offsets, dec_offsets]).T

            try:
                hull = ConvexHull(points)
                hit_area = hull.area
            except:
                hit_area = 0

            avg_beam_size = subset['beam_size_arcsec'].mean()
            beam_area = np.pi * (avg_beam_size / 2)**2
            threshold_area = area_multiplier * beam_area

            if hit_area > threshold_area:
                df.loc[subset.index, 'rfi_flag_local'] = True
                local_flag_count += len(subset)

    return df, local_flag_count



df_3_4_5_beam['rfi_flag_local'] = False
df_3_4_5_beam, local_count = flag_local_rfi(df_3_4_5_beam)


In [121]:
#flagging frequencies and drift rates found in same place at dif times/ time persistent rfi 
def flag_temporal_persistence_rfi(df,
                                   frequency_tol=1.0,
                                   drift_tol=1.0,
                                   radius_multiplier=3,
                                   min_time_gap=10,  # seconds
                                   min_repeat_count=2,
                                   debug=False):
    """
    Flags signals as RFI if they appear at the same sky location (within 3x beam size),
    same freq/drift (within tolerance), at different times (≥ min_time_gap),
    and occur at least min_repeat_count times.
    """
    df = df.copy()
    df['rfi_flag_global'] = False

    df['freq_bin'] = (df['signal_frequency'] / frequency_tol).round().astype(int)
    df['drift_bin'] = (df['signal_drift_rate'] / drift_tol).round().astype(int)

    dec_mean = df['dec_degrees'].mean()
    df['ra_arcsec'] = df['ra_hours'] * 3600 * np.cos(np.deg2rad(dec_mean))
    df['dec_arcsec'] = df['dec_degrees'] * 3600

    flagged_indices = set()
    grouped = df.groupby(['freq_bin', 'drift_bin'])

    for (f_bin, d_bin), group in grouped:
        if len(group) < min_repeat_count:
            continue

        coords = group[['ra_arcsec', 'dec_arcsec']].values
        beam_sizes = group['beam_size_arcsec'].values
        times = group['tstart_datetime'].values
        tree = cKDTree(coords)

        for i in range(len(group)):
            r_i = radius_multiplier * beam_sizes[i]
            time_i = times[i]

            idx_neighbors = tree.query_ball_point(coords[i], r=r_i)
            repeat_times = []

            for j in idx_neighbors:
                if i == j:
                    continue
                time_j = times[j]
                time_diff = abs(pd.Timedelta(time_i - time_j).total_seconds())
                if time_diff >= min_time_gap:
                    repeat_times.append(group.index[j])

            if len(set(repeat_times)) >= (min_repeat_count - 1):
                flagged_indices.add(group.index[i])
                flagged_indices.update(repeat_times)

    df.loc[list(flagged_indices), 'rfi_flag_global'] = True
    return df, len(flagged_indices)

df_3_4_5_beam['tstart_datetime'] = Time(df_3_4_5_beam['tstart'], format='mjd').to_datetime()
df_3_4_5_beam['rfi_flag_global'] = False
df_3_4_5_beam, global_count = flag_temporal_persistence_rfi(df_3_4_5_beam)
print(f"Global RFI flagged: {global_count}")



Global RFI flagged: 113630


In [122]:
# Group by freq/drift and count unique FOVs (file_uri) broader global rfi search 
repeat_groups = df_3_4_5_beam.groupby(['signal_frequency', 'signal_drift_rate'])['file_uri'].nunique()

# Find freq/drift pairs appearing in more than one distinct FOV
suspect_freqs = repeat_groups[repeat_groups > 1].index

# Mark those rows in the dataframe
df_3_4_5_beam['global_freq_repeat'] = df_3_4_5_beam.set_index(['signal_frequency', 'signal_drift_rate']).index.isin(suspect_freqs)

# Increase flag_strength for these broad repeats
df_3_4_5_beam.loc[df_3_4_5_beam['global_freq_repeat'], 'flag_strength'] += 2

print(f"Temporal persistence global RFI flagged: {(df_3_4_5_beam['rfi_flag_global']).sum()}")
print(f"Multi-FOV freq+drift repeats flagged: {df_3_4_5_beam['global_freq_repeat'].sum()}")

Temporal persistence global RFI flagged: 113630
Multi-FOV freq+drift repeats flagged: 43232


In [123]:
#set all flags within the df 
def apply_rfi_flagging(df, local_count=None, global_count=None,
                       apply_spatial_flags=True,
                       apply_power_ratio_flag=True,
                       power_ratio_threshold=1.0,
                       debug=False):
    if local_count is not None:
        print(f"Local RFI flagged:  {local_count}")
    if global_count is not None:
        print(f"Global RFI flagged: {global_count}")

    # Conditions
    cond1 = (df['rfi_flag_local']) & (~df['freq_has_incoherent_counterpart']) if apply_spatial_flags else pd.Series(False, index=df.index)
    cond2 = (df['signal_power'] > power_ratio_threshold * df['signal_incoherent_power'])
    cond2_only = cond2 & (~cond1)
    cond3 = df['freq_has_phase_center_counterpart']
    global_only = (df['rfi_flag_global']) & (~df['rfi_flag_local']) if apply_spatial_flags else pd.Series(False, index=df.index)

    df['rfi_flag'] = False
    df['flag_strength'] = 0

    if apply_spatial_flags:
        df.loc[df['rfi_flag_local'], 'rfi_flag'] = True
        df.loc[df['rfi_flag_local'], 'flag_strength'] += 1
        df.loc[global_only, 'rfi_flag'] = True
        df.loc[global_only, 'flag_strength'] += 1

    df.loc[cond1, 'flag_strength'] += 2
    df.loc[cond2_only, 'rfi_flag'] = True
    df.loc[cond2_only, 'flag_strength'] += 1
    df.loc[df['global_freq_repeat'], 'rfi_flag'] = True
    df.loc[df['global_freq_repeat'], 'flag_strength'] += 2
    df.loc[cond3, 'flag_strength'] += 1

    if debug:
        print("Flag strength distribution:")
        print(df['flag_strength'].value_counts().sort_index())
        print("Counts for conditions:")
        print(f"cond1 (local + no incoherent): {cond1.sum()}")
        print(f"cond2_only (coherent >> incoherent): {cond2_only.sum()}")
        print(f"global_only (global but not local): {global_only.sum()}")
        print(f"global_freq_repeat: {df['global_freq_repeat'].sum()}")
        print(f"cond3 (phase center): {cond3.sum()}")

        # EXTRA: For cases with apply_spatial_flags=False, print cond1 and cond2 counts separately
        if not apply_spatial_flags:
            print(f"cond1 total (should be zero): {cond1.sum()}")  # likely zero here
            print(f"cond2 total: {cond2.sum()}")

    return df



In [125]:
#split the pickle file into rfi levels for future analysis 
def split_by_flag_strength(df, clean_thresh=0, maybe_thresh=1):
    """
    Split dataframe into clean, maybe RFI, and strong RFI based on flag_strength thresholds.

    - clean_thresh: max flag_strength for 'clean' data (default 0)
    - maybe_thresh: max flag_strength for 'maybe' RFI data (default 1)
    """
    clean_df = df[df['flag_strength'] <= clean_thresh]
    maybe_rfi_df = df[(df['flag_strength'] > clean_thresh) & (df['flag_strength'] <= maybe_thresh)]
    strong_rfi_df = df[df['flag_strength'] > maybe_thresh]
    return clean_df, maybe_rfi_df, strong_rfi_df


df_3_4_5_beam = apply_rfi_flagging(df_3_4_5_beam, local_count=local_count, global_count=global_count, apply_spatial_flags=True, power_ratio_threshold=2, debug=False)

if 'global_freq_repeat' not in df_1_2_beam.columns:
    df_1_2_beam['global_freq_repeat'] = False
df_1_2_beam = apply_rfi_flagging(df_1_2_beam,apply_spatial_flags=False,power_ratio_threshold=1.25,debug=False)

# Combine datasets
fin_df = pd.concat([df_1_2_beam, df_3_4_5_beam], ignore_index=True)

# Split based on flag strength
clean_df, maybe_rfi_df, strong_rfi_df = split_by_flag_strength(fin_df)

# Print summary counts
print(f"Total detections: {len(fin_df)}")
print(f"Likely clean (flag_strength ≤ 0): {len(clean_df)}")
print(f"Maybe RFI (flag_strength == 1):   {len(maybe_rfi_df)}")
print(f"Strong RFI (flag_strength > 1):   {len(strong_rfi_df)}")


Local RFI flagged:  90345
Global RFI flagged: 113630


Total detections: 804791
Likely clean (flag_strength ≤ 0): 216940
Maybe RFI (flag_strength == 1):   511677
Strong RFI (flag_strength > 1):   76174


In [85]:
def flag_burst_rfi(df, burst_window=1.0, min_beams=3):
    df = df.copy()
    df['burst_rfi'] = False
    df['timestamp'] = df['tstart_datetime'].astype(np.int64) // 1_000_000_000  # seconds
    group = df.groupby(['file_uri', 'signal_frequency', 'timestamp'])
    for _, g in group:
        if g['signal_beam'].nunique() >= min_beams:
            df.loc[g.index, 'burst_rfi'] = True
    return df


In [ ]:
clean_df.to_pickle('data/clean_df.pkl')
maybe_rfi.to_pickle('data/maybe_rfi.pkl')
strong_rfi.to_pickle('data/strong_rfi.pkl')

In [42]:
from IPython.display import clear_output
clear_output(wait=True)